# 전처리 1

In [1]:
import pandas as pd
import numpy as np
import re
import copy
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline
from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords
pd.options.mode.chained_assignment = None  # 경고 숨기기

In [2]:
# 데이터 불러오기
news = pd.read_csv('naver_drop_duplicates.csv')
news

,Title,Date
0,"네이버, 금지·혐오 표현 기준 개정…12일부터 정책 적용",2023.06.01
1,"네이버, 초거대 AI로 소상공인 라이브커머스 대본 작성 돕는 이유",2023.06.01
2,네이버 악성 댓글땐 ‘악플러 꼬리표’ 단다,2023.06.01
3,네이버가 4달간 중소상공인 브랜드 구축 돕는다,2023.06.01
4,네이버·카카오 인물정보에 '장례지도사' 등 5개 직업 추가,2023.06.01
...,...,...
9340,"티몬, 브랜드 단위 협업으로 직구 강화…""큐텐 시너지로 차별화""",2024.05.27
9341,"카카오·라인야후, 어떻게 해커에 뚫렸나… “오픈채팅방 ID 구조 단순”vs“...",2024.05.27
9342,네이버웹툰 ‘꿈에서 자유로’ 오는 6월 두 번째 오디오 웹툰 텀블럭 펀딩 실...,2024.05.27
9343,“삼성 출신 들어가니 좀 달라졌네”…요즘 국회는 ‘AI 열공’,2024.05.27


### 1. Date 컬럼 형식 맞추기 - '실제 주식 데이터'와 비교 위해서

In [3]:
### 1. Date 컬럼 형식 맞추기 - '실제 주식 데이터'와 비교 위해서
news['Date'] = pd.to_datetime(news['Date'], format='%Y.%m.%d').dt.strftime('%Y-%m-%d')
# 컬럼 순서 변경
news = news[['Date', 'Title']]
news

,Date,Title
0,2023-06-01,"네이버, 금지·혐오 표현 기준 개정…12일부터 정책 적용"
1,2023-06-01,"네이버, 초거대 AI로 소상공인 라이브커머스 대본 작성 돕는 이유"
2,2023-06-01,네이버 악성 댓글땐 ‘악플러 꼬리표’ 단다
3,2023-06-01,네이버가 4달간 중소상공인 브랜드 구축 돕는다
4,2023-06-01,네이버·카카오 인물정보에 '장례지도사' 등 5개 직업 추가
...,...,...
9340,2024-05-27,"티몬, 브랜드 단위 협업으로 직구 강화…""큐텐 시너지로 차별화"""
9341,2024-05-27,"카카오·라인야후, 어떻게 해커에 뚫렸나… “오픈채팅방 ID 구조 단순”vs“..."
9342,2024-05-27,네이버웹툰 ‘꿈에서 자유로’ 오는 6월 두 번째 오디오 웹툰 텀블럭 펀딩 실...
9343,2024-05-27,“삼성 출신 들어가니 좀 달라졌네”…요즘 국회는 ‘AI 열공’


# 2. 텍스트 정제된 ppc_Title 컬럼 만들기

In [4]:
### 2. 텍스트 정제
def preprocess_title(text):  
    text = text.upper()  # (1) 대문자로 변환
    text = re.sub(r'\[.*?\]', ' ', text)   # (2) 대괄호 및 그 안의 내용 제거
    text = re.sub(r'[^\w\s]', ' ', text)  # (3) 특수 문자 제거 및 공백 유지
    text = text.strip() # (4) 문자열 앞뒤 공백 제거
    return text
# 'Title' 컬럼에 텍스트 정제 함수 적용
news['ppc_Title'] = news['Title'].apply(preprocess_title)

In [5]:
news

,Date,Title,ppc_Title
0,2023-06-01,"네이버, 금지·혐오 표현 기준 개정…12일부터 정책 적용",네이버 금지 혐오 표현 기준 개정 12일부터 정책 적용
1,2023-06-01,"네이버, 초거대 AI로 소상공인 라이브커머스 대본 작성 돕는 이유",네이버 초거대 AI로 소상공인 라이브커머스 대본 작성 돕는 이유
2,2023-06-01,네이버 악성 댓글땐 ‘악플러 꼬리표’ 단다,네이버 악성 댓글땐 악플러 꼬리표 단다
3,2023-06-01,네이버가 4달간 중소상공인 브랜드 구축 돕는다,네이버가 4달간 중소상공인 브랜드 구축 돕는다
4,2023-06-01,네이버·카카오 인물정보에 '장례지도사' 등 5개 직업 추가,네이버 카카오 인물정보에 장례지도사 등 5개 직업 추가
...,...,...,...
9340,2024-05-27,"티몬, 브랜드 단위 협업으로 직구 강화…""큐텐 시너지로 차별화""",티몬 브랜드 단위 협업으로 직구 강화 큐텐 시너지로 차별화
9341,2024-05-27,"카카오·라인야후, 어떻게 해커에 뚫렸나… “오픈채팅방 ID 구조 단순”vs“...",카카오 라인야후 어떻게 해커에 뚫렸나 오픈채팅방 ID 구조 단순 VS
9342,2024-05-27,네이버웹툰 ‘꿈에서 자유로’ 오는 6월 두 번째 오디오 웹툰 텀블럭 펀딩 실...,네이버웹툰 꿈에서 자유로 오는 6월 두 번째 오디오 웹툰 텀블럭 펀딩 실
9343,2024-05-27,“삼성 출신 들어가니 좀 달라졌네”…요즘 국회는 ‘AI 열공’,삼성 출신 들어가니 좀 달라졌네 요즘 국회는 AI 열공


# 3. ppc_Title 컬럼 중복 확인 및 제거

In [6]:
# 중복 확인
print(news[news.duplicated(subset='ppc_Title')==True].count())
news[news.duplicated(subset='ppc_Title', keep=False)].head()

# => 중복 원인
# Title을 전처리 하면서 동일한 내용이 됨

Date         24
Title        24
ppc_Title    24
dtype: int64


,Date,Title,ppc_Title
1242,2023-07-17,"""해운대 우수상품 '네이버 쇼핑 라이브'에서 만나요""",해운대 우수상품 네이버 쇼핑 라이브 에서 만나요
1261,2023-07-18,[Who Is ?] 이해진 네이버 글로벌투자책임자,이해진 네이버 글로벌투자책임자
1435,2023-07-24,해운대 우수상품 ‘네이버 쇼핑 라이브’에서 만나요,해운대 우수상품 네이버 쇼핑 라이브 에서 만나요
2983,2023-09-19,"DGB금융그룹, 네이버 해피빈 '이달의 기부'로 추석맞이 아동 지원",DGB금융그룹 네이버 해피빈 이달의 기부 로 추석맞이 아동 지원
2992,2023-09-19,"DGB금융그룹, 네이버 해피빈 ‘이달의 기부’로 추석맞이 아동 지원",DGB금융그룹 네이버 해피빈 이달의 기부 로 추석맞이 아동 지원


In [7]:
### 3. 중복 제거
news = news.drop_duplicates(subset='ppc_Title', keep='first')

In [8]:
# 중복 제거 확인
print(news[news.duplicated(subset='ppc_Title')==True].count())
news[news.duplicated(subset='ppc_Title')==True]

Date         0
Title        0
ppc_Title    0
dtype: int64


,Date,Title,ppc_Title


# 4. 띄어쓰기 기준 명사만 추출하여 Tokens 컬럼 만들기

In [9]:
### 4. 띄어쓰기 후 각 단어 형태소 분석 -> 명사만 추출
kiwi = Kiwi(typos='basic')
stopwords = Stopwords()

def tokenize_text(text):
    words = text.split()  # (1) 띄어쓰기로 텍스트 나누기
    token_list = []
    for word in words:
        tokens = kiwi.tokenize(word)  # (2) 각 단어를 형태소 분석
        for token in tokens:
            if token.tag in ['NNG', 'NNP']:   # (3) 명사(NNG, NNP)만 추출
                token_list.append(token.form)
    return token_list
news['Tokens'] = news['ppc_Title'].apply(tokenize_text)

In [ ]:
news.head(50)

,Date,Title,ppc_Title,Tokens
0,2023-06-01,"네이버, 금지·혐오 표현 기준 개정…12일부터 정책 적용",네이버 금지 혐오 표현 기준 개정 12일부터 정책 적용,"[네이버, 금지, 혐오, 표현, 기준, 개정, 정책, 적용]"
1,2023-06-01,"네이버, 초거대 AI로 소상공인 라이브커머스 대본 작성 돕는 이유",네이버 초거대 AI로 소상공인 라이브커머스 대본 작성 돕는 이유,"[네이버, 거대, 상공인, 라이브, 커머스, 대본, 작성, 이유]"
2,2023-06-01,네이버 악성 댓글땐 ‘악플러 꼬리표’ 단다,네이버 악성 댓글땐 악플러 꼬리표 단다,"[네이버, 악성, 댓글, 때, 악플, 꼬리표]"
3,2023-06-01,네이버가 4달간 중소상공인 브랜드 구축 돕는다,네이버가 4달간 중소상공인 브랜드 구축 돕는다,"[네이버, 중소, 상공인, 브랜드, 구축]"
4,2023-06-01,네이버·카카오 인물정보에 '장례지도사' 등 5개 직업 추가,네이버 카카오 인물정보에 장례지도사 등 5개 직업 추가,"[네이버, 카카오, 인물, 정보, 장례, 지도사, 직업, 추가]"
5,2023-06-01,"현대차證 “네이버, 하이퍼클로바X 시장 연착륙할 것”",현대차證 네이버 하이퍼클로바X 시장 연착륙할 것,"[현대차, 네이버, 하이퍼클로바, 시장, 연착륙]"
6,2023-06-01,"재난상황 ‘주연’된 네이버...공공인프라 된 플랫폼, 책임 어디까지?",재난상황 주연 된 네이버 공공인프라 된 플랫폼 책임 어디까지,"[재난, 상황, 주연, 네이버, 공공, 인프라, 플랫폼, 책임]"
7,2023-06-01,"AI 웹툰 논란에 네이버·카카오 ""인간 작품만 받는다""",AI 웹툰 논란에 네이버 카카오 인간 작품만 받는다,"[웹툰, 논란, 네이버, 카카오, 인간, 작품]"
8,2023-06-01,네이버가 300억 투자한 패션 플랫폼 ‘브랜디’ 자금 수혈 필요,네이버가 300억 투자한 패션 플랫폼 브랜디 자금 수혈 필요,"[네이버, 투자, 패션, 플랫폼, 브랜디, 자금, 수혈, 필요]"
9,2023-06-01,"네이버 마비, 5분 만에 복구됐지만… 실제 재난에선 어쩌나",네이버 마비 5분 만에 복구됐지만 실제 재난에선 어쩌나,"[네이버, 마비, 복구, 재난]"


# <span style="color:pink"> --- 중간 저장 -------------

In [ ]:
# CSV 파일로 저장
news.to_csv('naver_01.Preprocessing_01.csv', index=False)

# 5. 각 행의 리스트를 문자열로 결합하고 컴마 제거


In [ ]:
import pandas as pd 
news_copy = pd.read_csv('naver_01.Preprocessing_01.csv')

In [ ]:
# Tokens 열을 문자열로 변환
news_copy['Tokens'] = news_copy['Tokens'].astype(str)

In [ ]:
# ### 5. Tokens 열에서 리스트 기호와 쉼표 제거
# news['Tokens'] = news['Tokens'].str.replace("[\[\]']", "", regex=True)
# news['Tokens'] = news['Tokens'].str.replace(",", "", regex=False)
# news

In [ ]:
### 5. Tokens 열에서 리스트 기호와 쉼표 제거
news_copy['Tokens'] = news_copy['Tokens'].str.replace("[\[\]']", "", regex=True)
news_copy['Tokens'] = news_copy['Tokens'].str.replace(",", "", regex=False)
news_copy

,Date,Title,ppc_Title,Tokens
0,2023-06-01,"네이버, 금지·혐오 표현 기준 개정…12일부터 정책 적용",네이버 금지 혐오 표현 기준 개정 12일부터 정책 적용,네이버 금지 혐오 표현 기준 개정 정책 적용
1,2023-06-01,"네이버, 초거대 AI로 소상공인 라이브커머스 대본 작성 돕는 이유",네이버 초거대 AI로 소상공인 라이브커머스 대본 작성 돕는 이유,네이버 거대 상공인 라이브 커머스 대본 작성 이유
2,2023-06-01,네이버 악성 댓글땐 ‘악플러 꼬리표’ 단다,네이버 악성 댓글땐 악플러 꼬리표 단다,네이버 악성 댓글 때 악플 꼬리표
3,2023-06-01,네이버가 4달간 중소상공인 브랜드 구축 돕는다,네이버가 4달간 중소상공인 브랜드 구축 돕는다,네이버 중소 상공인 브랜드 구축
4,2023-06-01,네이버·카카오 인물정보에 '장례지도사' 등 5개 직업 추가,네이버 카카오 인물정보에 장례지도사 등 5개 직업 추가,네이버 카카오 인물 정보 장례 지도사 직업 추가
...,...,...,...,...
9316,2024-05-27,"티몬, 브랜드 단위 협업으로 직구 강화…""큐텐 시너지로 차별화""",티몬 브랜드 단위 협업으로 직구 강화 큐텐 시너지로 차별화,티몬 브랜드 단위 협업 직구 강화 큐 텐 시너지 차별
9317,2024-05-27,"카카오·라인야후, 어떻게 해커에 뚫렸나… “오픈채팅방 ID 구조 단순”vs“...",카카오 라인야후 어떻게 해커에 뚫렸나 오픈채팅방 ID 구조 단순 VS,카카오 라인 야후 해커 오픈 채팅방 구조 단순
9318,2024-05-27,네이버웹툰 ‘꿈에서 자유로’ 오는 6월 두 번째 오디오 웹툰 텀블럭 펀딩 실...,네이버웹툰 꿈에서 자유로 오는 6월 두 번째 오디오 웹툰 텀블럭 펀딩 실,네이버 웹툰 꿈 자유 오디오 웹툰 텀 블럭 펀딩 실
9319,2024-05-27,“삼성 출신 들어가니 좀 달라졌네”…요즘 국회는 ‘AI 열공’,삼성 출신 들어가니 좀 달라졌네 요즘 국회는 AI 열공,삼성 출신 요즘 국회


In [ ]:
# CSV 파일로 저장
news_copy.to_csv('naver_01.Preprocessing_02.csv', index=False)

---

# 6. 결측치 확인 및 제거

In [ ]:
import pandas as pd 
news_check = pd.read_csv('naver_01.Preprocessing_02.csv')
news_check.head()


,Date,Title,ppc_Title,Tokens
0,2023-06-01,"네이버, 금지·혐오 표현 기준 개정…12일부터 정책 적용",네이버 금지 혐오 표현 기준 개정 12일부터 정책 적용,네이버 금지 혐오 표현 기준 개정 정책 적용
1,2023-06-01,"네이버, 초거대 AI로 소상공인 라이브커머스 대본 작성 돕는 이유",네이버 초거대 AI로 소상공인 라이브커머스 대본 작성 돕는 이유,네이버 거대 상공인 라이브 커머스 대본 작성 이유
2,2023-06-01,네이버 악성 댓글땐 ‘악플러 꼬리표’ 단다,네이버 악성 댓글땐 악플러 꼬리표 단다,네이버 악성 댓글 때 악플 꼬리표
3,2023-06-01,네이버가 4달간 중소상공인 브랜드 구축 돕는다,네이버가 4달간 중소상공인 브랜드 구축 돕는다,네이버 중소 상공인 브랜드 구축
4,2023-06-01,네이버·카카오 인물정보에 '장례지도사' 등 5개 직업 추가,네이버 카카오 인물정보에 장례지도사 등 5개 직업 추가,네이버 카카오 인물 정보 장례 지도사 직업 추가


In [ ]:
news_check[news_check['Tokens'].isna()]

,Date,Title,ppc_Title,Tokens
5619,2024-01-03,1559호를 읽고[독자의 소리],1559호를 읽고,NaN
8928,2024-05-10,[연합뉴스 이 시각 헤드라인] - 14:30,14 30,NaN


In [ ]:
### 6. 결측치 제거
news = news.dropna(subset=['Tokens'])
news[news['Tokens'].isna()]

,Date,Title,ppc_Title,Tokens


In [ ]:
### 6. 결측치 제거
news_check = news_check.dropna(subset=['Tokens'])
news_check[news_check['Tokens'].isna()]

,Date,Title,ppc_Title,Tokens


In [ ]:
# CSV 파일로 저장
news_check.to_csv('naver_01.Preprocessing_FIN.csv', index=False)

---

In [ ]:
# 잘 저장 됐는지 확인
import pandas as pd 
news_check2 = pd.read_csv('naver_01.Preprocessing_FIN.csv')

In [ ]:
news_check2[news_check2['Tokens'].isna()]

,Date,Title,ppc_Title,Tokens


In [ ]:
news_check2.head(60)

,Date,Title,ppc_Title,Tokens
0,2023-06-01,"네이버, 금지·혐오 표현 기준 개정…12일부터 정책 적용",네이버 금지 혐오 표현 기준 개정 12일부터 정책 적용,네이버 금지 혐오 표현 기준 개정 정책 적용
1,2023-06-01,"네이버, 초거대 AI로 소상공인 라이브커머스 대본 작성 돕는 이유",네이버 초거대 AI로 소상공인 라이브커머스 대본 작성 돕는 이유,네이버 거대 상공인 라이브 커머스 대본 작성 이유
2,2023-06-01,네이버 악성 댓글땐 ‘악플러 꼬리표’ 단다,네이버 악성 댓글땐 악플러 꼬리표 단다,네이버 악성 댓글 때 악플 꼬리표
3,2023-06-01,네이버가 4달간 중소상공인 브랜드 구축 돕는다,네이버가 4달간 중소상공인 브랜드 구축 돕는다,네이버 중소 상공인 브랜드 구축
4,2023-06-01,네이버·카카오 인물정보에 '장례지도사' 등 5개 직업 추가,네이버 카카오 인물정보에 장례지도사 등 5개 직업 추가,네이버 카카오 인물 정보 장례 지도사 직업 추가
5,2023-06-01,"현대차證 “네이버, 하이퍼클로바X 시장 연착륙할 것”",현대차證 네이버 하이퍼클로바X 시장 연착륙할 것,현대차 네이버 하이퍼클로바 시장 연착륙
6,2023-06-01,"재난상황 ‘주연’된 네이버...공공인프라 된 플랫폼, 책임 어디까지?",재난상황 주연 된 네이버 공공인프라 된 플랫폼 책임 어디까지,재난 상황 주연 네이버 공공 인프라 플랫폼 책임
7,2023-06-01,"AI 웹툰 논란에 네이버·카카오 ""인간 작품만 받는다""",AI 웹툰 논란에 네이버 카카오 인간 작품만 받는다,웹툰 논란 네이버 카카오 인간 작품
8,2023-06-01,네이버가 300억 투자한 패션 플랫폼 ‘브랜디’ 자금 수혈 필요,네이버가 300억 투자한 패션 플랫폼 브랜디 자금 수혈 필요,네이버 투자 패션 플랫폼 브랜디 자금 수혈 필요
9,2023-06-01,"네이버 마비, 5분 만에 복구됐지만… 실제 재난에선 어쩌나",네이버 마비 5분 만에 복구됐지만 실제 재난에선 어쩌나,네이버 마비 복구 재난


In [ ]:
def find_sentiment_words(tokens, word_dict):
    pos_words = []
    neg_words = []
    zero_words = []
    sentiment_words = []

    total_polarity = 0

    for token in tokens.split():  # 각 단어를 공백 기준으로 분리
        if token in word_dict:
            sentiment_words.append(token)
            if word_dict[token] > 0:
                pos_words.append(token)
                total_polarity += word_dict[token]
            elif word_dict[token] < 0:
                neg_words.append(token)
                total_polarity += word_dict[token]
            else:
                zero_words.append(token)
        else:
            zero_words.append(token)

    return sentiment_words, pos_words, neg_words, zero_words, total_polarity

# 예제 데이터프레임과 word_dict를 사용하여 결과를 계산
sentiment_results = news['Stop_tokens'].apply(lambda x: find_sentiment_words(x, word_dict))

# 결과 출력
sentiment_results